In [1]:
# Предусмотреть возможность перераспределения классов. Так как классов может быть равное количество, не будет необходимости обучать несколько моделей.
# При этом важно отметить, что может быть наборот. Какой-то из классов имеет меньший обхем и это нужно определять.

# Нужны ограничения на количество обучаемых моделей. Если отношение слишком велико? Может ли быть такое?

''' Как в итоге работает решение?

1. Я обучаю каждый раз модели логрега на основе идентификатора пользователя. Это можно делать ночью. Нужно ли это делать каждый раз? Да, так как новости меняются. Исследовать.
2. На выходе полученной функции я получаю словарь моделей, который можно переиспользовать в боте, как часть алгоритма получения валидации новости для каждого пользователя.
3. Если аппроксимируемый класс модели == 1, то новость выводить, она положительная. Если == 0, то не выводить, так как она отрицальная. 
4. Нужно мониторить результат обучения модели. Возомжно стоит определить, когда модели нужно начинать обучаться. Спустя какое количество классов?

'''

' Как в итоге работает решение?\n\n1. Я обучаю каждый раз модели логрега на основе идентификатора пользователя. Это можно делать ночью. Нужно ли это делать каждый раз? Да, так как новости меняются. Исследовать.\n2. На выходе полученной функции я получаю словарь моделей, который можно переиспользовать в боте, как часть алгоритма получения валидации новости для каждого пользователя.\n3. Если аппроксимируемый класс модели == 1, то новость выводить, она положительная. Если == 0, то не выводить, так как она отрицальная. \n4. Нужно мониторить результат обучения модели. Возомжно стоит определить, когда модели нужно начинать обучаться. Спустя какое количество классов?\n\n'

In [2]:
import json
from ddl import cos_distance
import numpy as np
import os

import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import KFold

from collections import Counter

In [4]:
import pickle

In [5]:
user_id = 141587836
sources = ["rbc", "lenta", "ria", "habr"]
data_vectors = {}
matching_cos_distance = []
negative_samples = []
potential_positive_samples = []

In [6]:
# Загружаем все новости в оперативку
for where_from in sources:
    with open(f'./data/vectors/vectors_news_{where_from}.json', 'r') as f:
        temp = json.load(f)
        for i in temp:
            data_vectors[i] = temp[i]

In [7]:
# def find_intersections(news_title, user_id, news_title_emb):

# Открываем все реакции человека на новости

with open(f'./data/users/{user_id}/embeddings_{user_id}.json', 'r') as f:
    data = json.load(f)

# Находим отличия от отрицальных новостей 
negative_set = [id for id in data.keys() if data[id]['target'] == '0']
all_set = set(data_vectors.keys())
not_negative_set = all_set ^ set(negative_set)

print(len(negative_set), len(not_negative_set))

# Сравнение потенциальных положительных новостей со всеми новостями, поиск самых близких негативных к ним
c = 0
for id in negative_set:
    for id_all in not_negative_set:
        matching_cos_distance.append(cos_distance(np.array(data_vectors[id]['news_embedding']), np.array(data_vectors[id_all]['news_embedding'])))
        potential_positive_samples.append(id_all)

        c += 1
        if c % 1_000_000 == 0:
            print(c, end = '\r')


714 22767


In [8]:
# Определяем позитивные новости из существующей разметки пользователя
positive_samples = []
positive_set = [id for id in data.keys() if data[id]['target'] == '1']

negative_list_std = []

# Считаем отклонение для определения параметра отбора позитивных новостей, которые понравились
matching_std = np.std(matching_cos_distance)
matching_mean = np.mean(matching_cos_distance)

# Ищем позитивные для человека новости, потенциально это все новости, что меньше (среднее по всем сравнениям + 1 стандартное отклонение)
for item in zip(potential_positive_samples, matching_cos_distance):
    if item[1] >= (matching_mean + 2 * matching_std):
        negative_list_std.append(item[0])

In [9]:
# Вычитаем из позитивных потенциальных негативные относительно стандартного отклонения
positive_samples = set(potential_positive_samples) - set(negative_list_std)
# Расширяем сет размеченных позитивных позитивными потенциально
positive_set.extend(positive_samples)

In [10]:
# Сколько позитивных у нас получилось
print(len(positive_set))
# Проверяем, есть ли у нас пересечения между позитивными и негативными (не должно быть совсем)
print(len(set(negative_list_std) & set(positive_set)))

4157
5


In [11]:
# Получаем эмбеддинги каждого позитивного и негативного случая
positive_embeddings = []
negative_embeddings = []

for id in positive_set:
    positive_embeddings.append(data_vectors[id]['news_embedding'])

for id in set(negative_list_std):
    negative_embeddings.append(data_vectors[id]['news_embedding'])

In [12]:
# Превращаем в numpy все вектора позитивных и отрциательных новостей

positive_embeddings_array = np.array(positive_embeddings)
negative_embeddings_array = np.array(negative_embeddings)

In [13]:
# Смотрим какой shape для каждого
positive_embeddings_array.shape, negative_embeddings_array.shape

((4157, 300), (18624, 300))

In [14]:
# Заполняем 0 и 1 каждый из отрицательных и положительных новостей
ones_array = np.ones((positive_embeddings_array.shape[0],1))
zeros_array = np.zeros((negative_embeddings_array.shape[0],1))

In [15]:
# Собираем все воедино в датасет
a = np.hstack((positive_embeddings_array, ones_array))
b = np.hstack((negative_embeddings_array, zeros_array))
X_np = np.vstack((a, b))

In [16]:
# Затем пытаемся обучить модель искать такие или относить к негативным все остальные новости

In [17]:
X_df = pd.DataFrame(X_np).rename({300:'target'}, axis = 1)

# Получаем количество сэмплов для разделения наибольшего класса

n_splits = X_df['target'].value_counts()[0] // X_df['target'].value_counts()[1]

X_df['splits'] = X_df.index % n_splits

In [18]:
# Убираем дисбаланс, обучая на нескольких сэмплах несколько логистических регрессий
dict_models = {}

for i in range(n_splits - 1): # Оставляем последнюю часть для валидации
    target_0 = X_df[(X_df['target'] == 0) & (X_df['splits'] == i)].sample(n = X_df['target'].value_counts()[1], axis = 0)
    target_1 = X_df[X_df['target'] == 1]

    target_sampled = pd.concat([target_0, target_1], axis = 0)

    y = target_sampled['target']
    X = target_sampled.drop('target', axis = 1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    model = LogisticRegression(max_iter = 10000, verbose = 1, n_jobs = -1)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(classification_report(y_test, preds))

    # Сохраняем в словарь для сохранения в pickle
    dict_models[i] = model

    del model

c:\Users\roman\anaconda3\envs\news\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.8s finished
c:\Users\roman\anaconda3\envs\news\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\roman\anaconda3\envs\news\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


              precision    recall  f1-score   support

         0.0       0.89      0.92      0.91       832
         1.0       0.92      0.89      0.90       831

    accuracy                           0.90      1663
   macro avg       0.90      0.90      0.90      1663
weighted avg       0.90      0.90      0.90      1663



[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.7s finished
c:\Users\roman\anaconda3\envs\news\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\roman\anaconda3\envs\news\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87       832
         1.0       0.86      0.88      0.87       831

    accuracy                           0.87      1663
   macro avg       0.87      0.87      0.87      1663
weighted avg       0.87      0.87      0.87      1663



[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.3s finished


              precision    recall  f1-score   support

         0.0       0.89      0.86      0.87       832
         1.0       0.86      0.89      0.88       831

    accuracy                           0.88      1663
   macro avg       0.88      0.88      0.88      1663
weighted avg       0.88      0.88      0.88      1663



c:\Users\roman\anaconda3\envs\news\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [19]:
# На основе всех обученных логрегов учим предсказывать конечный результат c помощью еще одной логистической регресси

target_0 = X_df[(X_df['target'] == 0) & (X_df['splits'] == (n_splits - 1))].sample(n = X_df['target'].value_counts()[1], axis = 0)
target_1 = X_df[X_df['target'] == 1]

target_sampled = pd.concat([target_0, target_1], axis = 0)

y = target_sampled['target']
X = target_sampled.drop('target', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

logreg_number = 0
df_train_logregs = pd.DataFrame()
df_test_logregs = pd.DataFrame()

for model_id in dict_models:

    predicts_train = dict_models[model_id].predict(X_train)
    df_train_logregs[logreg_number] = predicts_train

    predicts_test = dict_models[model_id].predict(X_test)
    df_test_logregs[logreg_number] = predicts_test

    logreg_number += 1

model = LogisticRegression(max_iter = 10000, verbose = 1, n_jobs = -1)
model.fit(df_train_logregs, y_train)
preds = model.predict(df_test_logregs)
print(classification_report(y_test, preds))

dict_models['total'] = model
dict_models

c:\Users\roman\anaconda3\envs\news\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\roman\anaconda3\envs\news\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\roman\anaconda3\envs\news\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\roman\anaconda3\envs\news\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  war

              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91       832
         1.0       0.91      0.91      0.91       831

    accuracy                           0.91      1663
   macro avg       0.91      0.91      0.91      1663
weighted avg       0.91      0.91      0.91      1663



[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished


{0: LogisticRegression(max_iter=10000, n_jobs=-1, verbose=1),
 1: LogisticRegression(max_iter=10000, n_jobs=-1, verbose=1),
 2: LogisticRegression(max_iter=10000, n_jobs=-1, verbose=1),
 'total': LogisticRegression(max_iter=10000, n_jobs=-1, verbose=1)}

In [20]:
# Сохраняем все обученные модели

with open(f'./data/users/{user_id}/models.pickle', 'wb') as handle:
    pickle.dump(dict_models, handle, protocol=pickle.HIGHEST_PROTOCOL)